In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# copy application_df to a new dataframe
df = application_df.copy(deep=True)

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns=["EIN", "NAME"])

In [4]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = df.APPLICATION_TYPE.value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = df.CLASSIFICATION.value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])

In [11]:
# Split our preprocessed data into our features and target arrays
X = df.drop(columns=["IS_SUCCESSFUL"])
y = df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Identify numeric columns
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler to numeric columns
X_scaler = scaler.fit(X_train[numeric_columns])

# Scale the data
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Scale the data
X_train_scaled[numeric_columns] = X_scaler.transform(X_train[numeric_columns])
X_test_scaled[numeric_columns] = X_scaler.transform(X_test[numeric_columns])

###Compile, Train and Evaluate the Model

In [13]:
from keras.layers import Input

# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation="relu")) # changed the number of units from 80 to 128 units
nn.add(tf.keras.layers.Dropout(0.5)) # added a dropout layer of 50%
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu")) # changed the number of units from 30 to 64 units
nn.add(tf.keras.layers.Dropout(0.5)) # added a dropout layer of 50%
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu")) # added a third hidden layer with 32 units
nn.add(tf.keras.layers.Dropout(0.5)) # added a dropout layer of 5
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,001 (62.50 KB)

 Trainable params: 16,001 (62.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) # changed the learning rate from 0.01 to 0.0001
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=150) # changed the number of epochs from 100 to 150

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 508us/step - accuracy: 0.6301 - loss: 0.6504
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.7171 - loss: 0.5880
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.7272 - loss: 0.5776
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - accuracy: 0.7282 - loss: 0.5707
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 463us/step - accuracy: 0.7285 - loss: 0.5723
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.7281 - loss: 0.5714
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 0.7274 - loss: 0.5675
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.7285 - loss: 0.5648
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - accuracy: 0.7298 - loss: 0.5663
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.7343 - loss: 0.5585
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.7293 - loss: 0.5634
Epoch 12/150
804/80

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 472us/step - accuracy: 0.7289 - loss: 0.5538
Loss: 0.5538147687911987, Accuracy: 0.728863000869751


In [17]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")